In [3]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

# H2O 초기화
h2o.init()

# 데이터 로드 및 H2O 프레임으로 변환
data = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')
sampled_data = data.sample(frac=0.01, random_state = 42)
h2o_df = h2o.H2OFrame(sampled_data)

# Response 열을 범주형으로 변환
h2o_df['Response'] = h2o_df['Response'].asfactor()

# 데이터 분할
train, test = h2o_df.split_frame(ratios=[.8], seed=42)

# AutoML 모델 생성 및 학습
aml = H2OAutoML(max_runtime_secs=120, seed=42, sort_metric="AUC")
aml.train(y='Response', training_frame=train)

# 모델 평가
lb = aml.leaderboard
print(lb.head())

# 최적의 모델 정보 출력
best_model = aml.leader
print(best_model)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 day 20 hours 18 mins
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_jaesolshin_zvfdh3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.451 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse        mse
StackedEnsemble_BestOfFamily_2_AutoML_4_20240722_120302  0.860819   0.26379   0.377097                0.234424  0.293731  0.0862778
StackedEnsemble_AllModels_1_AutoML_4_20240722_120302     0.860681   0.263874  0.377871                0.254917  0.293856  0.0863516
GBM_2_AutoML_4_20240722_120302                           0.860082   0.264331  0.376579                0.230575  0.294001  0.0864369
StackedEnsemble_BestOfFamily_3_AutoML_4_20240722_120302  0.859814   0.264263  0.374801                0.236486  0.293996  0.0864337
StackedEnsemble_BestOfFamily_1_AutoML_4_20240722_120302  0.858692   0.265201  0.369813                0.242839  0.294587  0.0867817
GBM_1_AutoML_4_202

In [4]:
# test 데이터로 모델 평가
performance = best_model.model_performance(test)
print(performance)

ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.08668984679625112
RMSE: 0.2944313957380414
LogLoss: 0.26383261326535096
AUC: 0.8644462994502942
AUCPR: 0.39524641295356994
Gini: 0.7288925989005883
Null degrees of freedom: 23056
Residual degrees of freedom: 23052
Null deviance: 17409.012189934692
Residual deviance: 12166.377128118394
AIC: 12176.377128118394

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.23881099363947886
       0      1     Error    Rate
-----  -----  ----  -------  ----------------
0      16227  3939  0.1953   (3939.0/20166.0)
1      776    2115  0.2684   (776.0/2891.0)
Total  17003  6054  0.2045   (4715.0/23057.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.238811     0.47289   212
max f2                       0.118755     0.649921  302
max f0point5                 0.